# Step 2:
# Create Unique School and Daycare IDs and Assign Kids to Schools and Daycares

* Kids with age [0-3] go to Daycares
* Kids with age [4-11] go to Elementry school
* Kids with age [12-13] go to Middle school
* Kids with age [14-17] go to High school

In [13]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random
import timeit
#from shapely.ops import cascaded_union
import multiprocessing
from math import sin, cos, sqrt, atan2, radians
from src.tools import edu_to_gpd

## 1, Load Data

In [156]:
#Census Treact Boundary
#only read two columns
tract = gpd.read_file('data/Erie_Tract_WGS84.shp')#.loc[:,['GEOID10','geometry']]
tract.head()

,GEOID10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,DP0010001,DP0010002,DP0010003,DP0010004,...,DP0210001,DP0210002,DP0210003,DP0220001,DP0220002,DP0230001,DP0230002,Shape_Leng,Shape_Area,geometry
0,36029005300,Census Tract 53,2312977.0,5623.0,+42.9293726,-078.8597263,1458,34,17,24,...,454,165,289,377,397,2.28,1.37,0.067445,0.000256,"POLYGON ((-78.85173 42.92745, -78.85233 42.926..."
1,36029005400,Census Tract 54,1824964.0,105583.0,+42.9379527,-078.8681782,3850,191,177,179,...,1900,917,983,2166,1671,2.36,1.70,0.080981,0.000213,"POLYGON ((-78.87817 42.93603, -78.87819 42.936..."
2,36029005500,Census Tract 55,1551922.0,0.0,+42.9389099,-078.8889077,4054,358,301,216,...,1759,490,1269,1121,2916,2.29,2.30,0.072156,0.000171,"POLYGON ((-78.87819 42.93639, -78.87817 42.936..."
3,36029005600,Census Tract 56,2198732.0,0.0,+42.9511733,-078.8866130,4182,352,326,285,...,1782,653,1129,1409,2747,2.16,2.43,0.068333,0.000243,"POLYGON ((-78.89561 42.94638, -78.89534 42.947..."
4,36029005700,Census Tract 57,956845.0,0.0,+42.9497231,-078.8992508,2923,222,217,254,...,1168,426,742,1027,1891,2.41,2.55,0.048780,0.000106,"POLYGON ((-78.89683 42.94439, -78.89775 42.944..."


In [157]:
#Population (results from Step 1)
pop = pd.read_csv('../results/Erie_pop.csv').iloc[:,1:]
pop.head()

,id,age,sex,hhold,htype,wp,geometry,long,lat
0,36029000110i0,6.0,f,36029000110h335,5,NaN,POINT (-78.82896033753237 42.8319901673968),-78.828960,42.831990
1,36029000110i1,54.0,f,36029000110h66,9,36029010700w572,POINT (-78.82684360311679 42.844139551113386),-78.826844,42.844140
2,36029000110i2,27.0,m,36029000110h185,7,NaN,POINT (-78.85573318586734 42.839600789208994),-78.855733,42.839601
3,36029000110i3,13.0,f,36029000110h335,5,NaN,POINT (-78.82896033753237 42.8319901673968),-78.828960,42.831990
4,36029000110i4,28.0,m,36029000110h186,7,NaN,POINT (-78.83826068581105 42.85535764959998),-78.838261,42.855358


In [158]:
pop.shape

(900660, 9)

In [159]:
#Read Data
#Daycare
daycare = pd.read_csv('../results/daycares_erie_id.csv').iloc[:,1:]
dc_gdf = edu_to_gpd(daycare)
dc_gdf.head()

/Users/richard/PycharmProjects/pythonProject/ny-s-pop/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,ID,NAME,COUNTY,STATE,LATITUDE,LONGITUDE,ENROLLMENT,geometry,SchID,current
0,1932514220,Cao Head Start - St. Agatha'S,Erie,NY,42.856728,-78.827309,68.0,POINT (-78.82731 42.85673),36029000200d0,0
1,372414210,Valley Community Day Care,Erie,NY,42.868244,-78.851486,262.0,POINT (-78.85149 42.86824),36029000500d0,0
2,1932314204,Cao Head Start - Old First Ward,Erie,NY,42.866421,-78.862991,17.0,POINT (-78.86299 42.86642),36029000500d1,0
3,1953614220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836571,-78.807053,32.0,POINT (-78.80705 42.83657),36029000700d0,0
4,1953714220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836899,-78.807083,54.0,POINT (-78.80708 42.83690),36029000700d1,0


In [160]:
school = pd.read_csv('../results/schools_erie_id.csv').iloc[:,1:]
school_gdf = edu_to_gpd(school)
school_gdf.head()

/Users/richard/PycharmProjects/pythonProject/ny-s-pop/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,NCESID,NAME,COUNTY,STATE,LEVEL,LATITUDE,LONGITUDE,ENROLLMENT,geometry,SchID,current
0,360003604337,SOUTH BUFFALO CHARTER SCHOOL,ERIE COUNTY,NY,Primary,42.838200,-78.823990,671,POINT (-78.82399 42.83820),36029000110s0,0
1,360585000378,SOUTH PARK HIGH SCHOOL,ERIE COUNTY,NY,High,42.853061,-78.824250,773,POINT (-78.82425 42.85306),36029000200s0,0
2,360585000333,BILINGUAL CENTER,ERIE COUNTY,NY,Primary,42.868451,-78.848835,491,POINT (-78.84883 42.86845),36029000500s0,0
3,360585003478,DISCOVERY SCHOOL,ERIE COUNTY,NY,Primary,42.839878,-78.809253,567,POINT (-78.80925 42.83988),36029000700s0,0
4,360585000363,LORRAINE ELEMENTARY SCHOOL,ERIE COUNTY,NY,Primary,42.846954,-78.814273,597,POINT (-78.81427 42.84695),36029000800s0,0


## 2, Data Preprocess
Assign Unique IDs for Schools and Daycares

In [161]:
from src.tools import edu_to_gpd
from src.assign_workplace_edu import join_edu_data

### 2.1 Change to Spatial Data

### 2.2 Create Unique ID for Education Site
Locate the schools and daycares to assign unique IDs

### 2.3 Apply Spatial Join

In [162]:
#set census crs into wgs84
tract.crs = {'init' :'epsg:4326'}
census = tract.loc[:,['GEOID10','geometry']]

/Users/richard/PycharmProjects/pythonProject/ny-s-pop/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [163]:
dc_gdf.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [164]:
dc_gdf.head()

,ID,NAME,COUNTY,STATE,LATITUDE,LONGITUDE,ENROLLMENT,geometry,SchID,current
0,1932514220,Cao Head Start - St. Agatha'S,Erie,NY,42.856728,-78.827309,68.0,POINT (-78.82731 42.85673),36029000200d0,0
1,372414210,Valley Community Day Care,Erie,NY,42.868244,-78.851486,262.0,POINT (-78.85149 42.86824),36029000500d0,0
2,1932314204,Cao Head Start - Old First Ward,Erie,NY,42.866421,-78.862991,17.0,POINT (-78.86299 42.86642),36029000500d1,0
3,1953614220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836571,-78.807053,32.0,POINT (-78.80705 42.83657),36029000700d0,0
4,1953714220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836899,-78.807083,54.0,POINT (-78.80708 42.83690),36029000700d1,0


In [165]:
dc_gdf = join_edu_data(dc_gdf, census, "SchID", "Daycare")
dc_gdf.head()

/Users/richard/PycharmProjects/pythonProject/ny-s-pop/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,NAME,COUNTY,STATE,LATITUDE,LONGITUDE,ENROLLMENT,geometry,SchID,current,index_right,GEOID10,temp,count
0,1932514220,Cao Head Start - St. Agatha'S,Erie,NY,42.856728,-78.827309,68.0,POINT (-78.82731 42.85673),36029000200d0,0,92,36029000200,36029000200d,0
1,372414210,Valley Community Day Care,Erie,NY,42.868244,-78.851486,262.0,POINT (-78.85149 42.86824),36029000500d0,0,93,36029000500,36029000500d,0
2,1932314204,Cao Head Start - Old First Ward,Erie,NY,42.866421,-78.862991,17.0,POINT (-78.86299 42.86642),36029000500d1,0,93,36029000500,36029000500d,1
3,1953614220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836571,-78.807053,32.0,POINT (-78.80705 42.83657),36029000700d0,0,95,36029000700,36029000700d,0
4,1953714220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836899,-78.807083,54.0,POINT (-78.80708 42.83690),36029000700d1,0,95,36029000700,36029000700d,1


In [166]:
dc_gdf = dc_gdf.drop(columns=['index_right', 'GEOID10', 'temp', 'count'], axis=0)
dc_gdf['current'] = 0
dc_gdf.head()

,ID,NAME,COUNTY,STATE,LATITUDE,LONGITUDE,ENROLLMENT,geometry,SchID,current
0,1932514220,Cao Head Start - St. Agatha'S,Erie,NY,42.856728,-78.827309,68.0,POINT (-78.82731 42.85673),36029000200d0,0
1,372414210,Valley Community Day Care,Erie,NY,42.868244,-78.851486,262.0,POINT (-78.85149 42.86824),36029000500d0,0
2,1932314204,Cao Head Start - Old First Ward,Erie,NY,42.866421,-78.862991,17.0,POINT (-78.86299 42.86642),36029000500d1,0
3,1953614220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836571,-78.807053,32.0,POINT (-78.80705 42.83657),36029000700d0,0
4,1953714220,"Kiddie Corner Child Care Center, Inc.",Erie,NY,42.836899,-78.807083,54.0,POINT (-78.80708 42.83690),36029000700d1,0


In [167]:
school_gdf = join_edu_data(school_gdf, census, "SchID", "School")

/Users/richard/PycharmProjects/pythonProject/ny-s-pop/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [168]:
school_gdf = school_gdf.drop(columns=['index_right', 'GEOID10', 'temp', 'count'], axis=0)
school_gdf['current'] = 0
school_gdf.head()

,NCESID,NAME,COUNTY,STATE,LEVEL,LATITUDE,LONGITUDE,ENROLLMENT,geometry,SchID,current
0,360003604337,SOUTH BUFFALO CHARTER SCHOOL,ERIE COUNTY,NY,Primary,42.838200,-78.823990,671,POINT (-78.82399 42.83820),36029000110s0,0
1,360585000378,SOUTH PARK HIGH SCHOOL,ERIE COUNTY,NY,High,42.853061,-78.824250,773,POINT (-78.82425 42.85306),36029000200s0,0
2,360585000333,BILINGUAL CENTER,ERIE COUNTY,NY,Primary,42.868451,-78.848835,491,POINT (-78.84883 42.86845),36029000500s0,0
3,360585003478,DISCOVERY SCHOOL,ERIE COUNTY,NY,Primary,42.839878,-78.809253,567,POINT (-78.80925 42.83988),36029000700s0,0
4,360585000363,LORRAINE ELEMENTARY SCHOOL,ERIE COUNTY,NY,Primary,42.846954,-78.814273,597,POINT (-78.81427 42.84695),36029000800s0,0


In [72]:
#save to .csv file
school_gdf.to_csv('../results/schools_erie_id.csv')
dc_gdf.to_csv('../results/daycares_erie_id.csv')

## 3, Test fot Assign Daycare ID and School ID

In [48]:
#Apply Convert function
from src.tools import pop_to_geo_df
pop_gdf = pop_to_geo_df(pop)
#Add a column called tract indicating which census tract that the kid lives in
pop_gdf['tract'] = pop_gdf['hhold'].str[0:11]
pop_gdf.head()

/Users/richard/PycharmProjects/pythonProject/ny-s-pop/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,id,age,sex,hhold,htype,wp,geometry,long,lat,tract
0,36029000110i0,6.0,f,36029000110h335,5,NaN,POINT (-78.82896 42.83199),-78.828960,42.831990,36029000110
1,36029000110i1,54.0,f,36029000110h66,9,36029010700w572,POINT (-78.82684 42.84414),-78.826844,42.844140,36029000110
2,36029000110i2,27.0,m,36029000110h185,7,NaN,POINT (-78.85573 42.83960),-78.855733,42.839601,36029000110
3,36029000110i3,13.0,f,36029000110h335,5,NaN,POINT (-78.82896 42.83199),-78.828960,42.831990,36029000110
4,36029000110i4,28.0,m,36029000110h186,7,NaN,POINT (-78.83826 42.85536),-78.838261,42.855358,36029000110


### 3.1 Function to calculate the great circle distance
The reasons why we create this funtion is due the distance function from geopandas package is not giving us the right result. To fix the issue, we create a new distance function.

If yours works fine, you don't have to do this.

In [78]:
from src.assign_workplace_edu import new_distance

In [151]:
def assign_school(data, school, daycare):
    print(data)
    #if pd.isnull(test.loc[i,'wp']):
    if pd.isnull(data.wp):
        #print("wp is null")
        if data.age > 17:
            #print("work from home")
            #print
            return data.hhold
        #indi_buffer = data.geometry.buffer(0.08)
        #print(type(indi_buffer))
        #Assign edu site based on age
        elif data.age >= 4 and data.age <= 17:
            #print("school")
            return find_eduid(data.lat, data.long, data.geometry, school)

        elif data.age < 4:
            #print("daycare")
            return find_eduid(data.lat, data.long, data.geometry, daycare)

    else:
        return data.wp


In [152]:
def find_eduid (x, y, position, edu_site):
    buff = position.buffer(0.08)
    s_in = edu_site[edu_site.intersects(buff)].copy()

    if len(s_in) == 0:
        buff = position.buffer(0.2)
        s_in = edu_site[edu_site.intersects(buff)].copy()

        #Intersect Road Point Lat list
        sx = s_in.loc[:,'LATITUDE'].tolist()
        #Intersect Road Point Long list
        sy = s_in.loc[:, 'LONGITUDE'].tolist()

        #Calculate Distance between the point and intersected road points
        dist = [] #distance list
        for j in range(0, len(s_in)):
            d = new_distance(x, y, sx[j], sy[j])
            dist.append(d)

        #school ID list
        sid = s_in.loc[:,'SchID']

        #Create DF to hold the School ID an the their distance to Kids
        df_sch_in = pd.DataFrame({'SchID': sid, 'Dist':dist}).sort_values(by='Dist')#.reset_index(drop=True)
        #print("++++")
        #print(df_sch_in)

        sch_AgeDistAccept = [s for s in df_sch_in.index if edu_site.loc[s, 'current'] < edu_site.loc[s, 'ENROLLMENT']]
        #print(sch_AgeDistAccept)

        if sch_AgeDistAccept != []:
            j = sch_AgeDistAccept[0]
            #print(df_sch_in.loc[j, 'SchID'])
            return df_sch_in.loc[j, 'SchID']
        else:
            return random.choice(sid)

        school.loc[sch_id, 'current'] += 1

    else:
        # Intersect Road Point Lat list
        sx = s_in.loc[:, 'LATITUDE'].tolist()
        # Intersect Road Point Long list
        sy = s_in.loc[:, 'LONGITUDE'].tolist()

        # Calculate Distance between the point and intersected road points
        dist = []  # distance list
        for j in range(0, len(s_in)):
            d = new_distance(x, y, sx[j], sy[j])
            dist.append(d)

        # school ID list
        sid = s_in.loc[:, 'SchID']

        # Create DF to hold the School ID an the their distance to Kids
        df_sch_in = pd.DataFrame({'SchID': sid, 'Dist': dist}).sort_values(by='Dist')  # .reset_index(drop=True)
        # print("++++")
        # print(df_sch_in)

        sch_AgeDistAccept = [s for s in df_sch_in.index if edu_site.loc[s, 'current'] < edu_site.loc[s, 'ENROLLMENT']]
        # print(sch_AgeDistAccept)

        if sch_AgeDistAccept != []:
            j = sch_AgeDistAccept[0]
            # print(df_sch_in.loc[j, 'SchID'])
            return df_sch_in.loc[j, 'SchID']
        else:
            return random.choice(sid)

        school.loc[sch_id, 'current'] += 1

In [154]:
test = pop_gdf[:5].copy()
test

,id,age,sex,hhold,htype,wp,geometry,long,lat,tract
0,36029000110i0,6.0,f,36029000110h335,5,NaN,POINT (-78.82896 42.83199),-78.828960,42.831990,36029000110
1,36029000110i1,54.0,f,36029000110h66,9,36029010700w572,POINT (-78.82684 42.84414),-78.826844,42.844140,36029000110
2,36029000110i2,27.0,m,36029000110h185,7,NaN,POINT (-78.85573 42.83960),-78.855733,42.839601,36029000110
3,36029000110i3,13.0,f,36029000110h335,5,NaN,POINT (-78.82896 42.83199),-78.828960,42.831990,36029000110
4,36029000110i4,28.0,m,36029000110h186,7,NaN,POINT (-78.83826 42.85536),-78.838261,42.855358,36029000110


In [155]:
test.index

RangeIndex(start=0, stop=5, step=1)

In [147]:
test['wp'] = test.apply(assign_school,args=(school_gdf, dc_gdf),axis=1)
test

id                                        36029000110i0
age                                                 6.0
sex                                                   f
hhold                                   36029000110h335
htype                                                 5
wp                                                  NaN
geometry    POINT (-78.82896033753237 42.8319901673968)
long                                          -78.82896
lat                                            42.83199
tract                                       36029000110
Name: 0, dtype: object
id                                          36029000110i1
age                                                  54.0
sex                                                     f
hhold                                      36029000110h66
htype                                                   9
wp                                        36029010700w572
geometry    POINT (-78.82684360311679 42.844139551113386)
long       

,id,age,sex,hhold,htype,wp,geometry,long,lat,tract
0,36029000110i0,6.0,f,36029000110h335,5,36029000110s0,POINT (-78.82896 42.83199),-78.828960,42.831990,36029000110
1,36029000110i1,54.0,f,36029000110h66,9,36029010700w572,POINT (-78.82684 42.84414),-78.826844,42.844140,36029000110
2,36029000110i2,27.0,m,36029000110h185,7,36029000110h185,POINT (-78.85573 42.83960),-78.855733,42.839601,36029000110
3,36029000110i3,13.0,f,36029000110h335,5,36029000110s0,POINT (-78.82896 42.83199),-78.828960,42.831990,36029000110
4,36029000110i4,28.0,m,36029000110h186,7,36029000110h186,POINT (-78.83826 42.85536),-78.838261,42.855358,36029000110


### 3.2 Assign School and DaycareID function

In [127]:
print('Start running...')
#set timer
start_time = timeit.default_timer()
pop_gdf['wp'] = pop_gdf.apply(assign_school,args=(school_gdf, dc_gdf),axis=1)
elapsed = timeit.default_timer() - start_time
print("Total Time(s):", elapsed)
print('End program')

Start running...


KeyboardInterrupt: 

In [48]:
pop_gdf.shape

(907959, 9)

In [49]:
final_pop.to_csv('../results/ERIE_pop_w_id.csv')

### 3.3 Apply Assign School ID funtion

In [ ]:
def assign_ID(data):
    data['wp'] = data.apply(assign_school,args=(census, sch_df),axis=1)
    return data

def parallelize(data, func):
    #Cores
    num_cores = 7
    num_partitions = num_cores
    #Split
    data_split = np.array_split(data, num_partitions)
    #Pools
    pool = multiprocessing.Pool(num_cores)
    
    df = pd.concat(pool.map(func, data_split))
    return df

    pool.close()
    pool.join()
    
if __name__=='__main__':
    print('Start running...')
    #set timer
    start_time = timeit.default_timer()
    
    #apply
    g_sch_k['tract'] = g_sch_k['hhold'].str[0:11]
    print(g_sch_k.head())
    
    n_g_sch_k = parallelize(g_sch_k, assign_ID)
    
    elapsed = timeit.default_timer() - start_time
    print("Total Time(s):", elapsed)
    print('End program')